## Import Packages and Expert Data Set

In [16]:
import pickle
import chess
import pandas as pd
import numpy as np 
from chess_minimax import get_minimax_move
from chess_random import get_random_move

In [17]:
with open('2014_08_over_2400_chess_data.pickle','rb') as read_file:
    chess_df = pickle.load(read_file)

chess_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106 entries, Game47674 to Game993220
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   BlackElo       106 non-null    object
 1   WhiteElo       106 non-null    object
 2   moves          106 non-null    object
 3   move_num       106 non-null    object
 4   winner         106 non-null    object
 5   Black_Elo_Num  106 non-null    int64 
 6   White_Elo_Num  106 non-null    int64 
dtypes: int64(2), object(5)
memory usage: 6.6+ KB


<br>

## Data Cleaning and Transformation

In [18]:
# Create variable for just moves
moves = chess_df['moves']

In [19]:
# Transform data set to individual moves per row 

# List of characters to remove from moves 
char_list = []
for i in range(1,500):
    char_list.append(str(i) + '.')

move_list = []

for game in moves:
    index = list(moves).index(game)
    allmoves = game.split()
    all_moves = [elem for elem in allmoves if elem not in char_list]
    board = chess.Board()
    for i in range(len(all_moves)):
        board.push_san(all_moves[i])
        fen = board.fen()
        row = [index, i, fen, all_moves[i]]
        move_list.append(row)

In [20]:
# Turn list into dataframe
move_df = pd.DataFrame(move_list, columns = ['Game_Num', 'Move_Num', 'FEN', 'Move'])

# Create variable for next move
move_df['Next_Move'] = move_df.Move.shift(-1)

# Remove last move from each game
ddf = move_df.groupby(['Game_Num'])['Move_Num', 'FEN', 'Move', 'Next_Move'].apply(lambda x : x[:-1])

<ipython-input-20-508f604a7da8>:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  ddf = move_df.groupby(['Game_Num'])['Move_Num', 'FEN', 'Move', 'Next_Move'].apply(lambda x : x[:-1])


<br>

## Run Functions and Evaluate the Moves Matched

In [21]:
for index, row in ddf.iterrows():
    m0 = get_random_move(row['FEN'])
    ddf.loc[index,'Rand_Move'] = m0
    m1 = get_minimax_move(row['FEN'])
    ddf.loc[index,'Minimax_Move'] = m1

ddf.head()

KeyboardInterrupt: 

In [ ]:
ddf['Random_Compare'] = np.where(ddf['Next_Move'] == ddf['Rand_Move'], True, False)
ddf['Minimax_Compare'] = np.where(ddf['Next_Move'] == ddf['Minimax_Move'], True, False)

In [ ]:
ddf['Random_Compare'].value_counts()

In [ ]:
ddf['Minimax_Compare'].value_counts()

<br>

## Look at Recommendation Performance by Stage of Game

In [ ]:
# Get number of total moves in the game, 1 is added because I removed the last move earlier
ddf['Total_Moves'] = ddf.groupby(['Game_Num'])['Move_Num'].transform(max) + 1

# Find percent of the way through the game
ddf['Percent_Through_Game'] = (ddf['Move_Num']+1)/ddf['Total_Moves']

# Classify as beginning, middle, or end 
ddf['Stage_Of_Game'] = np.where(ddf['Percent_Through_Game'] < .3334, 'Beginning',
                       np.where(ddf['Percent_Through_Game'] < .6667, 'Middle',         
                               'End'))
ddf.head()

In [ ]:
# Crosstab of Minimax moves matched by stage of game
pd.crosstab(ddf['Minimax_Compare'], ddf['Stage_Of_Game'])

In [ ]:
# Crosstab of Randome moves matched by stage of game
pd.crosstab(ddf['Random_Compare'], ddf['Stage_Of_Game'])

#### Both show improvement at end of game, likely because the pool of possible moves greatly diminishes